# Mini-Project: Artistic effects on images

This mini-project is part of the course *Digital Image Processing* given at the University of Fribourg, Automn Semester 2022. It consists of creating three different artistic effects on color images. These effects are:
1. Oil painting effect
2. Aging effect
3. Mosaic effect

**To run the Notebook, use the `Restart & Run All` button from the `Kernel` menu. At the end of the Notebook are some interactions to modify and test the artistic effects.**

---
### Libraries imports
---

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import copy
from sklearn.metrics.pairwise import euclidean_distances
import ipywidgets as widgets
from ipywidgets import interact

---
## 1) Oil painting effect
---

Covolution for a 3D image (e.g. RGB)

In [ ]:
def convolution3d(image, kernel):
    assert image.shape[0] == image.shape[1], "ATM image should be square for convolutions"
    img_dim = image.shape[1]
    kern_dim = kernel.shape[1]

    # We want the same output dimensions as the input
    out_dim = img_dim
    # Can calculate the wanted padding from the general formula
    # out_dim = int(np.floor((img_dim + 2*padding - kern_dim) / stride) + 1)
    # Where we set stride=1
    padding = int(((out_dim - 1) - img_dim + kern_dim) / 2)
    
    # Padding * 2 because we add zeros on width and length, but not height
    padded_img = np.zeros((img_dim + padding * 2, img_dim + padding * 2, 3))
    # Put image on the center
    if padding != 0:
        padded_img[padding:-padding, padding:-padding] = image
    else:
        padded_img = copy.deepcopy(image)
        
    convolved_img = np.zeros((out_dim, out_dim))
    # +1 because otherwise it is excluded (and we will have a black line)
    for i in range(len(padded_img) - kern_dim + 1):
        for j in range(len(padded_img[0]) - kern_dim + 1):
            # Convolution step
            # Element-wise multiplication, then sum
            convolved_img[i][j] = np.array(kernel * padded_img[i:i+kern_dim, j:j+kern_dim]).sum()
                    
    return convolved_img

Main function for oil painting, inspired from http://supercomputingblog.com/graphics/oil-painting-algorithm/

In [ ]:
def oil_painting(img_data, radius=5):
    # Intensity calculations
    intensity_values = convolution3d(img_data, np.ones((1, 1, 3))/3)
    
    # New reference in memory
    painting_img = copy.deepcopy(img_data)

    # Pixels on the edges are left unchanged
    for i in range(1, len(intensity_values)-1):
        for j in range(1, len(intensity_values[0])-1):
            # Radius will be reduced for pixel near the edges -> avoid that too much pixels are left unchanged
            # To get a local zone
            tmp_radius = radius
            local_window = intensity_values[i-tmp_radius:i+tmp_radius+1, j-tmp_radius:j+tmp_radius+1]
            while((local_window.shape[0] == 0) or (local_window.shape[1] == 0)):
                tmp_radius = tmp_radius - 1
                local_window = intensity_values[i-tmp_radius:i+tmp_radius+1, j-tmp_radius:j+tmp_radius+1]
            
            # Reference and count the different intensities
            unique, counts = np.unique(local_window, return_counts=True)
            most_occurent_intensity = unique[np.argmax(counts)]
            # Take only the value having the given intensity
            bool_mask = local_window == most_occurent_intensity
            # For each color channel
            sum_rgb = []
            for k in range(3):
                intensity_pixels = bool_mask*img_data[i-tmp_radius:i+tmp_radius+1, j-tmp_radius:j+tmp_radius+1, k]
                sum_rgb.append(int(intensity_pixels.sum()))
                        
            # New RGB value is the ponderated with the most occurent intensity count
            new_pixel = np.array(sum_rgb) / counts.max()
            painting_img[i][j] = new_pixel

    return painting_img

---
## 2) Aging effect
---

Functions to convert between RGB and HSL (hue, saturation, luminence) color spaces

In [ ]:
def rgb_to_hsl(rgb_data):
    # New reference, with floats instead of ints
    new_hsl = rgb_data.astype(np.dtype('float64'))

    for i, row in enumerate(rgb_data):
        for j, column in enumerate(row):
            # Normalize
            pixel = column / 255
            c_max = np.max(pixel)
            c_min = np.min(pixel)
            delta = c_max - c_min
            argmax = np.argmax(pixel)
            red, green, blue = pixel
            
            # Compute hue
            if delta == 0:
                hue = 0
            elif argmax == 0:
                hue = 60 * (((green - blue) / delta) % 6)
            elif argmax == 1:
                hue = 60 * (((blue - red) / delta) + 2)
            else:
                hue = 60 * (((red - green) / delta) + 4)
                
            # Compute lightness
            lightness = (c_max + c_min) / 2
                
            # Compute saturation
            if delta == 0:
                saturation = 0
            else:
                saturation = delta / (1 - np.abs(2 * lightness - 1))

            new_hsl[i][j] = np.array([hue, saturation, lightness])
            
    return new_hsl

In [ ]:
def hsl_to_rgb(hsl_data):
    # Integer needed in RGB
    new_rgb = hsl_data.astype(np.dtype('uint8'))

    for i, row in enumerate(hsl_data):
        for j, column in enumerate(row):
            hue, saturation, lightness = column
            
            # Compute the constants needed in the calculation for RGB
            c_const = (1 - np.abs(2 * lightness - 1)) * saturation
            x_const = c_const * (1 - np.abs((hue / 60) % 2 - 1))
            m_const = lightness - c_const / 2       
            
            # Assign RGB values depending on the hue
            if 0 <= hue and hue < 60:
                red = c_const
                green = x_const
                blue = 0
            elif 60 <= hue and hue < 120:
                red = x_const
                green = c_const
                blue = 0
            elif 120 <= hue and hue < 180:
                red = 0
                green = c_const
                blue = x_const
            elif 180 <= hue and hue < 240:
                red = 0
                green = x_const
                blue = c_const
            elif 240 <= hue and hue < 300:
                red = x_const
                green = 0
                blue = c_const
            elif 300 <= hue and hue < 360:
                red = c_const
                green = 0
                blue = x_const
                
            pixel = np.array([red, green, blue])
            pixel = (pixel + m_const) * 255
            # Integers needed in RGB
            new_rgb[i][j] = pixel.astype(np.dtype('uint8'))
    
    return new_rgb

Automatically normalize an input value in an input range to another output range

In [ ]:
def range_normalization(value, min_input, max_input, min_output, max_output):
    return (value - min_input) / (max_input - min_input) * (max_output - min_output) + min_output

This custom distribution returns 80% of the time the same input value. Otherwise it samples from a random distribution below or above the input value with equal probability

In [ ]:
def custom_distribution(value, fluctuation):
    rand = np.random.rand()
    if rand < 0.8:
        return value
    if rand < 0.9:
        return np.random.uniform(value - fluctuation, value)
    if rand < 1:
        return np.random.uniform(value + 1, value + fluctuation + 1)

Main function for the aging effect

In [ ]:
def aging(img_data, degree_lightness, border_width_factor):
    img_height, img_width, _nchannels = img_data.shape
    hsl_img = rgb_to_hsl(img_data)
    
    # New reference in memory
    aged_img = copy.deepcopy(hsl_img)
    
    # Lightness value for the borders of the image
    edge_lightness = 0.8
    border_width = int(border_width_factor * min(img_height, img_width))
    border_fluctuations = max(1, int(0.01 * border_width))
    
    # Generate the borders using the custom distribution defined above (to smooth the borders)
    # Left and right borders
    left_border_limits = [int(custom_distribution(border_width, border_fluctuations))]
    right_border_limits = [int(custom_distribution(img_width-border_width, border_fluctuations))]
    for i in range(img_height-1):
        left_border = left_border_limits[-1]
        left_border_limits.append(int(custom_distribution(left_border, border_fluctuations)))
        right_border = right_border_limits[-1]
        right_border_limits.append(int(custom_distribution(right_border, border_fluctuations)))
    # Top and bottom borders
    top_border_limits = [int(custom_distribution(border_width, border_fluctuations))]
    bottom_border_limits = [int(custom_distribution(img_height-border_width, border_fluctuations))]
    for j in range(img_width-1):
        top_border = top_border_limits[-1]
        top_border_limits.append(int(custom_distribution(top_border, border_fluctuations)))
        bottom_border = bottom_border_limits[-1]
        bottom_border_limits.append(int(custom_distribution(bottom_border, border_fluctuations)))

    for row_idx, row in enumerate(hsl_img):
        for col_idx, pixel in enumerate(row):
            # Red-yellow are between 0 and 60 degrees in hue
            # Here fixed parameters, but one could modify them
            new_hue = range_normalization(pixel[0], 0, 360, 32, 35)
            new_saturation = range_normalization(pixel[1], 0, 1, 0.2, 0.5)
            new_lightness = range_normalization(pixel[2], 0, 1, 0, 0.7)

            # Modify the lightness for the borders
            if col_idx < left_border_limits[row_idx]:
                new_lightness = edge_lightness + (new_lightness-edge_lightness) * (col_idx/left_border_limits[row_idx])**degree_lightness
            if col_idx > right_border_limits[row_idx]:
                new_lightness = new_lightness + (edge_lightness-new_lightness) * ((col_idx-right_border_limits[row_idx])/(img_width-right_border_limits[row_idx]))**(1/degree_lightness)
            if row_idx < top_border_limits[col_idx]:
                new_lightness = edge_lightness + (new_lightness-edge_lightness) * (row_idx/top_border_limits[col_idx])**degree_lightness
            if row_idx > bottom_border_limits[col_idx]:
                new_lightness = new_lightness + (edge_lightness-new_lightness) * ((row_idx-bottom_border_limits[col_idx])/(img_height-bottom_border_limits[col_idx]))**(1/degree_lightness)

            # Aggregate and place new pixel
            new_pixel = np.array([new_hue, new_saturation, new_lightness])
            aged_img[row_idx][col_idx] = new_pixel
    
    return hsl_to_rgb(aged_img)

---
## 3) Mosaic effect
---

Function to generate a Voronoi partition. Seeds are generated randomly, but not too *close* to each other

In [ ]:
def voronoi_partition(img, k, border):
    img_height, img_width, _nchannels = img.shape
    # Keys are 'black' or the seeds positions
    partitions = {'black': []}

    # Generate a first seed
    i = 0
    seeds = []
    first_seed = np.array([np.random.randint(0, img_height), np.random.randint(0, img_width)])
    seeds.append(first_seed)
    partitions[str(first_seed)] = [first_seed]
    
    # Set a minimum distance between seeds
    # to ensure seeds are not too close from each other
    # Approximation with k circles inside a rectangle
    min_seed_distance = np.sqrt((img_width*img_height)/(np.pi*k))
    # For border width
    border_tolerance = range_normalization(border, 0, 1, 0.2*min_seed_distance, 0.35*min_seed_distance)
    
    # Generate k random seeds
    while(i < k-1):
        seed = np.array([np.random.randint(0, img_height), np.random.randint(0, img_width)])
        # euclidean_distances returns a (1,k)-shaped array
        distances = euclidean_distances([seed], seeds)[0]
        if (distances > min_seed_distance).all():
            seeds.append(seed)
            partitions[str(seed)] = [seed]
            i += 1
    seeds = np.array(seeds)

    for i in range(img_height):
        for j in range(img_width):
            point = np.array([i,j])
            # euclidean_distances returns a (1,k)-shaped array
            distances = euclidean_distances([point], seeds)[0]

            sorted_distances = list(zip(distances, range(len(seeds))))
            sorted_distances.sort(key=lambda x: x[0])

            # If too close to two seeds, consider the pixel as black
            if np.abs(sorted_distances[0][0] - sorted_distances[1][0]) < border_tolerance:
                partitions['black'].append(point)
            else:
                # Take seed key of minimum element
                partition_key = str(seeds[sorted_distances[0][1]])
                partitions[partition_key].append(point)
    
    return partitions

In [ ]:
def mosaic(img, k, border):
    # New reference
    mosaic_img = copy.deepcopy(img)
    
    partitions = voronoi_partition(img, k, border)
    
    for key, partition in partitions.items():
        if key == 'black':
            for point in partition:
                i, j = point
                mosaic_img[i][j] = np.array([0,0,0])
            continue

        rgb_values = []
        for point in partition:
            i, j = point
            rgb_values.append(img[i,j])
            
        new_rgb_value = np.mean(rgb_values, axis=0)

        for point in partition:
            i, j = point
            mosaic_img[i][j] = new_rgb_value
            
    return mosaic_img

---
## Try the effects!
---

Select the image on the left and the effect to apply on the right

In [ ]:
root_path = Path('images')
images = [path.name for path in root_path.iterdir()]
img_radiobuttons = widgets.RadioButtons(options=images)
img_vbox = widgets.VBox(children=[img_radiobuttons])

effects = ['Oil Painting', 'Aging', 'Mosaic']
effects_radiobuttons = widgets.RadioButtons(options=effects)
effects_vbox = widgets.VBox(children=[effects_radiobuttons])

vboxes = [img_vbox, effects_vbox]
output = widgets.HBox(children=vboxes)
display(output)

Try different effects with the interactive cursors. The original image is shown on the left while the transformed image is shown on the right

In [ ]:
img_path = root_path / img_radiobuttons.value
img_data = np.asarray(Image.open(img_path))

# Plot the original image next to the transformed one
def double_img_plot(img1, img2):
    fig = plt.figure(figsize=(16,9))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img1)
    plt.axis('off')
    fig.add_subplot(1, 2, 2)
    plt.imshow(img2)
    plt.axis('off')

if effects_radiobuttons.value == 'Oil Painting':
    @interact(radius=(1, 10, 1))
    def plot_painting(radius=5):
        oiled_img = oil_painting(img_data, radius)
        double_img_plot(img_data, oiled_img);
        
elif effects_radiobuttons.value == 'Aging':
    @interact(lightness=(2, 4, 1), border_width=(0.05, 0.15, 0.02))
    def plot_aging(lightness=2, border_width=0.15):
        aged_img = aging(img_data, lightness, border_width)
        double_img_plot(img_data, aged_img);
        
elif effects_radiobuttons.value == 'Mosaic':
    @interact(n_cells=(100, 1000, 100), cell_border=(0, 1, 0.1))
    def plot_mosaic(n_cells=500, cell_border=0.5):
        mosaic_img = mosaic(img_data, n_cells, cell_border)
        double_img_plot(img_data, mosaic_img);